Sequence Read Archive (SRA) is a database that contains DNA sequencing data in terms of short reads generated by high-throughput sequencing obtained via different platforms. The sequences are usually less than 1,000 base pairs in length. The database is available at http://www.ncbi.nlm.nih.gov/sra. This recipe will introduce steps to query and access the data residing in the SRA database from within R.

SRA数据库，包含了不同平台获取的高通量测序产生的短DNA测序数据。这些序列的长度通常小雨1000个碱基对。该数据库可从http://www.ncbi.nlm.nih.gov/sra 获取。这个教程主要是从R中查询并获取SRA数据库中的数据。

1. First, install and load the SRAdb library in your R session as follows:

 安装并加载SRAdb

In [1]:
source("http://bioconductor.org/biocLite.R")
biocLite("SRAdb")
library(SRAdb)

Bioconductor version 3.7 (BiocInstaller 1.30.0), ?biocLite for help
A newer version of Bioconductor is available for this version of R,
  ?BiocUpgrade for help
BioC_mirror: https://bioconductor.org
Using Bioconductor 3.7 (BiocInstaller 1.30.0), R 3.5.1 (2018-07-02).
Installing package(s) 'SRAdb'
also installing the dependency 'GEOquery'



package 'GEOquery' successfully unpacked and MD5 sums checked
package 'SRAdb' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\Administrator\AppData\Local\Temp\RtmpcDQaS7\downloaded_packages


Old packages: 'ade4', 'ape', 'backports', 'BH', 'BiocManager', 'broom',
  'callr', 'caret', 'checkpoint', 'class', 'cli', 'clipr', 'codetools',
  'colorspace', 'curl', 'data.table', 'dbplyr', 'ddalpha', 'digest', 'dimRed',
  'doParallel', 'dplyr', 'evaluate', 'fansi', 'forcats', 'foreign', 'geometry',
  'ggplot2', 'haven', 'htmlwidgets', 'httpuv', 'httr', 'igraph', 'ipred',
  'IRdisplay', 'IRkernel', 'jsonlite', 'kernlab', 'knitr', 'later', 'lattice',
  'lava', 'magic', 'markdown', 'MASS', 'Matrix', 'mgcv', 'mime', 'MKmisc',
  'ModelMetrics', 'modelr', 'openssl', 'pillar', 'pkgconfig', 'pls',
  'processx', 'purrr', 'R6', 'Rcpp', 'readr', 'readxl', 'recipes', 'repr',
  'reprex', 'rlang', 'rmarkdown', 'robustbase', 'rstudioapi', 'RUnit',
  'scales', 'sfsmisc', 'shiny', 'stringi', 'stringr', 'survival', 'testthat',
  'tibble', 'tidyr', 'tidyselect', 'tinytex', 'TTR', 'xfun', 'XML', 'xtable',
  'xts', 'zoo'
Loading required package: RSQLite
Warning message:
"package 'RSQLite' was built und

2. Now, fetch the metadata file from the SRA server as follows (note that this might take a while; therefore, it may be a good idea to grab a coffee now):

 从SRA服务器中加载metadata文件，比较难下载，可以选择其他下载软件下载到目录中。

In [ ]:
sqlFle <- getSRAdbFile()#比较难下载，可以用其他软件下载后解压到当前文件夹

3. Create a connection to the SQLite file for querying, as follows:
 
 连接SQLite文件以便查询：

In [2]:
sraCon <- dbConnect(SQLite(), 'SRAmetadb.sqlite')

4. There are a few things you can do to investigate the content of the database.
This will help you to define the queries that should be run later on. Use the
following function to determine the fields in the data:

 确定这个SQLite文件数据库的内容。以便后面查询的时候可以运行：

In [3]:
sraTables <- dbListTables(sraCon)
dbListFields(sraCon,"study")

[1] "study_ID"             "study_alias"          "study_accession"     
 [4] "study_title"          "study_type"           "study_abstract"      
 [7] "broker_name"          "center_name"          "center_project_name" 
[10] "study_description"    "related_studies"      "primary_study"       
[13] "sra_link"             "study_url_link"       "xref_link"           
[16] "study_entrez_link"    "ddbj_link"            "ena_link"            
[19] "study_attribute"      "submission_accession" "sradb_updated"

5. Now, frame your first query to get the accessions and titles from the SRA study where,
the study type contains the keyword embryo. The query can be seen as a typical
SQL query

 构建第一个查询：以从SRA研究中获得accession和title，其中研究类型包括关键字。这是典型的SQL查询方式：

In [6]:
myHit <- dbGetQuery(sraCon, paste("select study_accession,study_title from study where","study_description like'%embryo'",sep=" ")) #
myHit

study_accession,study_title
SRP001353,GSE17621: RNA-Seq of Drosophila developmental stage 16-18 hr
ERP014605,Ultra-deep sequencing of ribosome-associated poly-adenylated RNA in early Drosophila embryo reveals hundreds of conserved translating sORFs


6. To do a free text search for the terms of interest, use the following functions:
 
  要对感兴趣的属于进行免费文本搜索，请使用以下函数：

In [7]:
myHit <- getSRA( search_terms = "brain", out_types =c('run','study'), sraCon)
head(myHit)

run_alias,run,run_date,updated_date,spots,bases,run_center,experiment_name,run_url_link,run_entrez_link,...,study_title,study_type,study_abstract,center_project_name,study_description,study_url_link,study_entrez_link,study_attribute,related_studies,primary_study
DRR000060,DRR000060,2008-09-25,2015-02-06,6219174,223890264,UT-MGS,DRX000023,NA,NA,...,Massive transcriptional start site mapping of human fetal brain cells.,Transcriptome Analysis,"Comprehensive identification and characterization of the transcriptional start sites of human genes were carried out. For this purpose, we used our TSS-Seq method, in which next gene sequencing technology and our full-length cDNA library technology, oligo-capping were combined.",Integrateve Transcriptome Analysis,"Although recent studies have revealed that the majority of human genes are subjected to regulation of alternative promoters (APs), the biological relevance of this phenomenon remains unclear. To enable more comprehensive TSS analysis in the respective cell types, we recently developed a method, combining oligo-capping with the massively paralleled sequencing technology, Illumina GA. In this method, which we named TSS Seq, sequence adaptor which is necessary for Illumina GA sequencing is directly introduced to the cap site of the mRNA. By sequencing 36-48 sequence immediately downstream of the TSSs (TSS tags), it is possible to obtain precise positional information of transcriptional start sites (TSSs). In this paper, we used the TSS tag data accumulated from twelve different cell types and normal tissues in humans for the identification and characterization of the APs in human genes.",DBTSS: http://dbtss.hgc.jp/,NA,NA,NA,NA
DRR000061,DRR000061,2008-09-25,2015-02-06,6082236,218960496,UT-MGS,DRX000023,NA,NA,...,Massive transcriptional start site mapping of human fetal brain cells.,Transcriptome Analysis,"Comprehensive identification and characterization of the transcriptional start sites of human genes were carried out. For this purpose, we used our TSS-Seq method, in which next gene sequencing technology and our full-length cDNA library technology, oligo-capping were combined.",Integrateve Transcriptome Analysis,"Although recent studies have revealed that the majority of human genes are subjected to regulation of alternative promoters (APs), the biological relevance of this phenomenon remains unclear. To enable more comprehensive TSS analysis in the respective cell types, we recently developed a method, combining oligo-capping with the massively paralleled sequencing technology, Illumina GA. In this method, which we named TSS Seq, sequence adaptor which is necessary for Illumina GA sequencing is directly introduced to the cap site of the mRNA. By sequencing 36-48 sequence immediately downstream of the TSSs (TSS tags), it is possible to obtain precise positional information of transcriptional start sites (TSSs). In this paper, we used the TSS tag data accumulated from twelve different cell types and normal tissues in humans for the identification and characterization of the APs in human genes.",DBTSS: http://dbtss.hgc.jp/,NA,NA,NA,NA
DRR000063,DRR000063,2008-09-25,2015-02-06,6062563,218252268,UT-MGS,DRX000023,NA,NA,...,Massive transcriptional start site mapping of human fetal brain cells.,Transcriptome Analysis,"Comprehensive identification and characterization of the transcriptional start sites of human genes were carried out. For this purpose, we used our TSS-Seq method, in which next gene sequencing technology and our full-length cDNA library technology, oligo-capping were combined.",Integrateve Transcriptome Analysis,"Although recent studies have revealed that the majority of human genes are subjected to regulation of alternative promoters (APs), the biological relevance of this phenomenon remains unclear. To enable more comprehensive TSS analysis in the respective cell types, we recently developed a method, combining oligo-capping with the massively paralleled sequencing techno

7. Combine key words for appropriate logic as follows:
 
 结合关键词以及逻辑词进行查询：

In [8]:
myHit <- getSRA( search_terms ='Alzheimers OR "EPILEPSY"', out_types = c('sample'), sraCon)

8. Take a look at some of the retrieved results by typing the following command:

 查看检索（查询）结果：

In [9]:
head(myHit)

sample_alias,sample,taxon_id,common_name,anonymized_name,individual_name,description,sample_url_link,sample_entrez_link,sample_attribute
